<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 56.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=50f4da70c2d27392a139bb388c3f6bdeb85bceac51a4e72916b3d9cbce9aea7e
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# import libraries

In [7]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

In [8]:
# Start session

In [9]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("SparkML Ops").getOrCreate()

In [10]:
# import spark Ml libraries

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [12]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [13]:
!wget -N "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"

--2022-11-23 12:10:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.007s  

2022-11-23 12:10:12 (30.8 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [14]:
# Load the csv into a spark dataframe

In [15]:
searchterms = pd.read_csv('searchterms.csv')
sdf = spark.createDataFrame(searchterms)

In [16]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [17]:
row_count = sdf.count()
col_count = len(sdf.columns)
print(f'Total number of rows: {row_count}')
print(f'Total number of columns: {col_count}')

Total number of rows: 10000
Total number of columns: 4


In [18]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [19]:
print(sdf.show(5))

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows

None


In [20]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [21]:
sdf.dtypes

[('day', 'bigint'),
 ('month', 'bigint'),
 ('year', 'bigint'),
 ('searchterm', 'string')]

## OR

In [22]:
sdf.printSchema()

root
 |-- day: long (nullable = true)
 |-- month: long (nullable = true)
 |-- year: long (nullable = true)
 |-- searchterm: string (nullable = true)



In [23]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [24]:
sdf.createOrReplaceTempView("sdf")
spark.sql("""select count(*) as gaming_laptop from sdf where searchterm='gaming laptop'""").show()

+-------------+
|gaming_laptop|
+-------------+
|          499|
+-------------+



In [25]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [26]:
spark.sql("""select searchterm,count(*) from sdf group by searchterm order by count(*) desc limit 5""").show()

+-------------+--------+
|   searchterm|count(1)|
+-------------+--------+
|mobile 6 inch|    2312|
|    mobile 5g|    2301|
|mobile latest|    1327|
|       laptop|     935|
|  tablet wifi|     896|
+-------------+--------+



In [27]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [28]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [29]:
!wget -N "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.gzip"
!tar -xvzf model.gzip

--2022-11-23 12:10:56--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.gzip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [application/gzip]
Saving to: ‘model.gzip’

model.gzip          100%[===================>]   1.68K  --.-KB/s    in 0s      

2022-11-23 12:10:56 (261 MB/s) - ‘model.gzip’ saved [1722/1722]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_predi

In [30]:
# You need LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel

In [31]:
model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [32]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [33]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|176.14285712605306|
+------------------+

